# 20152410 배형준 머신러닝 과제9

In [1]:
# library import

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
# set my local working directory

import os

directory = 'C:\\Users\\golds\\Desktop\\중앙대학교\\2020-1 4학년 1학기\\머신러닝'
os.chdir(directory)

In [3]:
# load dataset

filename = './과제9/mnist.csv'
mnist = pd.read_csv(filename, header=None)
mnist.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,784
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
# convert data type from pd.DataFrame to np.array

label = np.array(mnist.iloc[:, 0]).reshape(-1, 1)
data = np.array(mnist.iloc[:, 1:])

## Implement Normalization class

행 방향으로 정규화 : 한 행에서 (하나의 숫자 그림에서) 가장 작은 값이 0, 가장 큰 값이 1이 되도록 변환

In [5]:
# make class 'minmaxscaler'

class minmaxscaler:
    
    def __init__(self):
        self.min_value = 0
        self.max_value = 0
        
    def fit(self, X):
        X = np.array(X)
        self.min_value = np.min(X, axis=1)
        self.max_value = np.where(np.max(X, axis=1) == 0, 1, np.max(X, axis=1))
        # 행 별 최대 최소, 열 방향으로
        
        return self
    
    def transform(self, X):
        X = np.array(X)
        scaled = np.zeros(X.shape)
        
        for j in range(X.shape[0]):
            scaled[j, :] = (X[j, :] - self.min_value[j]) / (self.max_value[j] - self.min_value[j])
        
        return scaled

In [6]:
minmax_scaler_model = minmaxscaler()
minmax_scaler_model.fit(data)
data_scaled = minmax_scaler_model.transform(data)

## Implement Onehotencoding class

In [7]:
class onehotencoding:
    
    def __init__(self):
        self.unique = 0
    
    def fit(self, X):
        X = np.array(X)
        self.unique = np.unique(X)
        
        return self
    
    def transform(self, X):
        X = np.array(X)
        m = X.shape[0]
        n = self.unique.shape[0]
    
        empty = np.zeros((m, n))
        
        for i in range(m):
            for j in range(n):
                if X[i] == self.unique[j]:
                    empty[i, j] = 1
        
        return empty

In [8]:
onehot_model = onehotencoding()
onehot_model.fit(label)
label_onehot = onehot_model.transform(label)

## Split trainset and testset

In [9]:
label_onehot_train = label_onehot[:6000, :]
label_onehot_test = label_onehot[6000:, :]

data_scaled_train = data_scaled[:6000, :]
data_scaled_test = data_scaled[6000:, :]

## Implement neural network

In [ ]:
class neural_network:

    def __init__(self, learning_rate, error_bound, iteration, hidden_layer, number_node, fit_intercept):
        self.learning_rate = learning_rate
        self.error_bound = error_bound
        self.iteration = iteration
        
        self.hidden_layer = hidden_layer # int
        self.number_node = number_node # list of int
        self.fit_intercept = fit_intercept # True or False
        
        self.coef_ = []
        self.record_train_cost = []
        self.record_test_cost = []
        self.record_train_accuracy = []
        self.record_test_accuracy = []
    
    def sigmoid(self, X, coef):
        z = np.dot(X, coef)
        sigmoid_value = 1 / (1 + np.exp(-z))
        
        return sigmoid_value
    
    def cost(self, X, coef_list, label):
        delta = 10**(-10)
        temp = X
        
        for coef in coef_list:
            sig = self.sigmoid(temp, coef)
            temp = sig
        
        cost_value = -np.mean(np.sum(label * np.log(sig + delta) + (1 - label) * np.log(1 - sig + delta), axis=1))
        
        return cost_value
    
    def predict(self, X, coef_list):
        temp = X
        
        for coef in coef_list:
            sig = self.sigmoid(temp, coef)
            temp = sig
        
        predict_value = np.argmax(sig, axis=1)
        
        return predict_value
    
    def predict_probability(self, X, coef_list):
        temp = X
        
        for coef in coef_list:
            sig = self.sigmoid(temp, coef)
            temp = sig
        
        return sig
    
    def fit(self, X_train, Y_train, X_test, Y_test): # Y_train, Y_test는 onehotencoding이 완료된 데이터
        X_train = np.array(X_train)
        Y_train = np.array(Y_train)
        m = X_train.shape[0]
        n = X_train.shape[1]
        q = X_test.shape[0]
        p = Y_train.shape[1]
        label_train = np.argmax(Y_train, axis=1) # 정확도를 확인하기 위한 기존 label
        label_test = np.argmax(Y_test, axis=1)
        
        self.number_node.insert(0, n)
        self.number_node.append(p)
        coef_ = []
        
        # fit_intercept
        if self.fit_intercept == True:
            X_train = np.column_stack(np.ones((m, 1)), X_train)
            X_test = np.column_stack(np.ones((q, 1)), X_test)
            
            number_node_with_intercept = []
            for number in self.number_node:
                number_node_with_intercept.append(number+1)
                
        else:
            number_node_with_intercept = self.number_node
        
        # set initial parameters
        for layer in range(self.hidden_layer + 1):
            temp_theta = np.random.randn(number_node_with_intercept[layer], self.number_node[layer+1])
            coef_.append(temp_theta)
        
        # model fitting
        while True:
            # calculate train and test cost
            train_cost_value = self.cost(X_train, coef_, Y_train)
            test_cost_value = self.cost(X_test, coef_, Y_test)
            
            self.record_train_cost.append(train_cost_value)
            self.record_test_cost.append(test_cost_value)
            
            # calculate train and test accuracy
            train_predict = self.predict(X_train, coef_)
            test_predict = self.predict(X_test, coef_)
            
            train_accuarcy = np.mean(train_predict == label_train)
            test_accuarcy = np.mean(test_predict == label_test)
            
            self.record_train_accuracy.append(train_accuarcy)
            self.record_test_accuracy.append(test_accuarcy)
            
            # calculate gradient using back propagation
            
            
            
            
            
            # renew the parameters
            
            
            
            
            
            # stopping rules
    
            
            
            
            
        
                
            
            
        
    
    
    
        
        
        

In [12]:
temp = np.array(np.ones((10000, 1)), data)
temp.shape

TypeError: data type not understood

In [15]:
temp = np.column_stack((np.ones((10000, 1)), data))
temp.shape

(10000, 785)

In [18]:
temp[:3, :50]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [19]:
a = [1, 2, 3]
a.insert(0, 9)
a

[9, 1, 2, 3]

In [24]:
np.argmax(label_onehot_train, axis=1)

array([7, 2, 1, ..., 5, 1, 7], dtype=int64)

In [26]:
a = np.argmax(label_onehot_train, axis=1)
a.shape

(6000,)

In [27]:
b = np.argmax(label_onehot_test, axis=1)
b.shape

(4000,)

In [28]:
np.mean(a[:4000] == b)

0.105